In [1]:
from pathlib import Path
import math
import time

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from tqdm.auto import tqdm

In [2]:
path = Path("./data/128x128/clean/E_clean_TG_Re6.0_NX128_NY128_NT101.npz")
data = np.load(path)

x = data["x"]
y = data["y"]
t = data["t"]

U = data["U"]
V = data["V"]
P = data["P"]

Nx, Ny, Nt = len(x), len(y), len(t)
Re = 6.0

assert U.shape == (Nt, Ny, Nx)
assert V.shape == (Nt, Ny, Nx)
assert P.shape == (Nt, Ny, Nx)

Tg, Yg, Xg = np.meshgrid(t, y, x, indexing="ij") # (Nt, Ny, Nx)

x_flat = Xg.ravel()
y_flat = Yg.ravel()
t_flat = Tg.ravel()

u_flat = U.ravel()
v_flat = V.ravel()
p_flat = P.ravel()

N_total = x_flat.size

rng = np.random.RandomState(0)
perm = rng.permutation(N_total)

n_train = int(0.7 * N_total)
n_val = int(0.15 * N_total)
n_test   = int(0.15 * N_total)

idx_train = perm[:n_train]
idx_val = perm[n_train:n_train+n_val]
idx_test  = perm[n_train+n_val:]

train = {k: v[idx_train] for k, v in {
    "x": x_flat, "y": y_flat, "tau": t_flat,
    "u": u_flat, "v": v_flat, "p": p_flat
}.items()}
val = {k: v[idx_val] for k, v in {
    "x": x_flat, "y": y_flat, "tau": t_flat,
    "u": u_flat, "v": v_flat, "p": p_flat
}.items()}
test = {k: v[idx_test] for k, v in {
    "x": x_flat, "y": y_flat, "tau": t_flat,
    "u": u_flat, "v": v_flat, "p": p_flat
}.items()}

print("train:", train["x"].shape[0],
      "val:", val["x"].shape[0],
      "test:", test["x"].shape[0])

train: 1158348 val: 248217 test: 248219


In [3]:
class TGDataset(Dataset):
    def __init__(self, data_dict):
        self.x = torch.from_numpy(data_dict["x"]).float()
        self.y = torch.from_numpy(data_dict["y"]).float()
        self.tau = torch.from_numpy(data_dict["tau"]).float()
        self.u = torch.from_numpy(data_dict["u"]).float()
        self.v = torch.from_numpy(data_dict["v"]).float()
        self.p = torch.from_numpy(data_dict["p"]).float()

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return {
            "x": self.x[idx],
            "y": self.y[idx],
            "tau": self.tau[idx],
            "u": self.u[idx],
            "v": self.v[idx],
            "p": self.p[idx],
        }


train_ds = TGDataset(train)
val_ds  = TGDataset(val)
test_ds  = TGDataset(test)

BATCH_SIZE = 4_096 
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader  = DataLoader(val_ds,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


batch = next(iter(train_loader))
print({k: v.shape for k, v in batch.items()})

{'x': torch.Size([4096]), 'y': torch.Size([4096]), 'tau': torch.Size([4096]), 'u': torch.Size([4096]), 'v': torch.Size([4096]), 'p': torch.Size([4096])}


In [4]:
class Sine(nn.Module):
    def __init__(self, w0=1.0):
        super().__init__()
        self.w0 = w0

    def forward(self, x):
        return torch.sin(self.w0 * x)


class TaylorGreenMLP(nn.Module):
    def __init__(self, in_dim=3, hidden_dim=128, n_hidden=6, w0=1.0):
        super().__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.n_hidden = n_hidden
        self.w0 = w0
        layers = []
        layers.append(nn.Linear(in_dim, hidden_dim))
        layers.append(Sine(w0))
        for _ in range(n_hidden - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(Sine(w0))
        self.backbone = nn.Sequential(*layers)
        self.head_psi = nn.Linear(hidden_dim, 1)
        self.head_p = nn.Linear(hidden_dim, 1)
        self._init_weights_siren()

    def _init_weights_siren(self):
        first_done = False
        for m in self.backbone:
            if isinstance(m, nn.Linear):
                if not first_done:
                    nn.init.uniform_(m.weight, -1.0/self.in_dim, 1.0/self.in_dim)
                    nn.init.zeros_(m.bias)
                    first_done = True
                else:
                    fan_in = m.in_features
                    bound = math.sqrt(6.0/fan_in)/self.w0
                    nn.init.uniform_(m.weight, -bound, bound)
                    nn.init.zeros_(m.bias)
        nn.init.kaiming_uniform_(self.head_psi.weight, a=0.0, nonlinearity="linear")
        nn.init.zeros_(self.head_psi.bias)
        nn.init.kaiming_uniform_(self.head_p.weight, a=0.0, nonlinearity="linear")
        nn.init.zeros_(self.head_p.bias)

    def forward(self, x):
        h = self.backbone(x)
        psi_hat = self.head_psi(h)
        p_hat = self.head_p(h)
        return psi_hat, p_hat


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TaylorGreenMLP(
    in_dim=3,
    hidden_dim=64, # нейроны
    n_hidden=8, # слои
    w0=1.0, # частота
).to(device)

# batch = next(iter(train_loader))
# inp = torch.stack(
#     [batch["x"], batch["y"], batch["tau"]],
#     dim=1
# ).to(device)
# psi_pred, p_pred = model(inp)
# print(psi_pred.shape, p_pred.shape)

In [5]:
a1 = 1.0
a2 = 1e-3
a3 = 0

# a2 = 1e-4
# a3 = 3e-7

MAX_A2 = 1e-2
MAX_A3 = 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.5)
num_epochs = 200

x_grid = torch.from_numpy(x).float().to(device)
y_grid = torch.from_numpy(y).float().to(device)
t_grid = torch.from_numpy(t).float().to(device)

Nx = x_grid.shape[0]
Ny = y_grid.shape[0]
Nt = t_grid.shape[0]

dx = float(x_grid[1] - x_grid[0])
dy = float(y_grid[1] - y_grid[0])
dt = float(t_grid[1] - t_grid[0])

# стена сдвигов, соответствующая дискретному выводу
# индекс 0 обязательно центр (i,j,k)
STENCIL = torch.tensor([
    [0, 0, 0],   # 0 центр
    [1, 0, 0],   # 1  (i+1, j, k)
    [-1, 0, 0],  # 2  (i-1, j, k)
    [2, 0, 0],   # 3  (i+2, j, k)
    [-2, 0, 0],  # 4  (i-2, j, k)
    [0, 1, 0],   # 5  (i, j+1, k)
    [0, -1, 0],  # 6  (i, j-1, k)
    [0, 2, 0],   # 7  (i, j+2, k)
    [0, -2, 0],  # 8  (i, j-2, k)
    [1, 1, 0],   # 9  (i+1, j+1, k)
    [1, -1, 0],  # 10 (i+1, j-1, k)
    [-1, 1, 0],  # 11 (i-1, j+1, k)
    [-1, -1, 0], # 12 (i-1, j-1, k)
    [0, 0, 1],   # 13 (i, j, k+1)
    [0, 0, -1],  # 14 (i, j, k-1)
    [1, 0, 1],   # 15 (i+1, j, k+1)
    [-1, 0, 1],  # 16 (i-1, j, k+1)
    [0, 1, 1],   # 17 (i, j+1, k+1)
    [0, -1, 1],  # 18 (i, j-1, k+1)
    [1, 0, -1],  # 19 (i+1, j, k-1)
    [-1, 0, -1], # 20 (i-1, j, k-1)
    [0, 1, -1],  # 21 (i, j+1, k-1)
    [0, -1, -1], # 22 (i, j-1, k-1)
], device=device)


# def pack_inputs(xb, yb, tb):
#     return torch.stack([xb, yb, tb], dim=1)


def coords_from_batch(xb, yb, tb):
    """
    xb, yb, tb: (B,)
    возвращает целочисленные индексы i,j,k по равномерной сетке
    """
    i = torch.round(xb / dx).long()
    j = torch.round(yb / dy).long()
    k = torch.round((tb - t_grid[0]) / dt).long()
    i = i.remainder(Nx)
    j = j.remainder(Ny)
    k = torch.clamp(k, 0, Nt - 1)
    return i, j, k


def eval_stencil(model, i, j, k):
    B = i.shape[0]
    S = STENCIL.shape[0]
    offs = STENCIL.unsqueeze(0).expand(B, -1, -1)  # (B, S, 3)
    i_off = (i.unsqueeze(1) + offs[:, :, 0]).remainder(Nx)
    j_off = (j.unsqueeze(1) + offs[:, :, 1]).remainder(Ny)
    k_off = (k.unsqueeze(1) + offs[:, :, 2])
    k_off = torch.clamp(k_off, 0, Nt - 1)
    x_off = x_grid[i_off]
    y_off = y_grid[j_off]
    t_off = t_grid[k_off]
    x_norm = x_off / (2 * math.pi)
    y_norm = y_off / (2 * math.pi)
    t_norm = (t_off - t_grid[0]) / (t_grid[-1] - t_grid[0])
    pts = torch.stack([x_norm, y_norm, t_norm], dim=2).view(B * S, 3)
    psi_all, p_all = model(pts)
    psi_all = psi_all.view(B, S)
    p_all = p_all.view(B, S)
    return psi_all, p_all, k_off


def time_derivative(f, f_tp, f_tm, k_int):
    """
    центральная разность по времени с односторонними на концах
    f, f_tp, f_tm: (B,)
    k_int: (B,) индексы времени текущих точек
    """
    df = (f_tp - f_tm) / (2.0 * dt)
    at_start = (k_int == 0)
    if at_start.any():
        df[at_start] = (f_tp[at_start] - f[at_start]) / dt
    at_end = (k_int == Nt - 1)
    if at_end.any():
        df[at_end] = (f[at_end] - f_tm[at_end]) / dt
    return df


def physics_forward(model, xb, yb, tb, u_true=None, v_true=None, details: bool = False):
    # индексы по сетке
    i, j, k_int = coords_from_batch(xb, yb, tb)
    # x_in = xb / (2 * math.pi)
    # y_in = yb / (2 * math.pi)
    #t_in = (tb - t_grid[0]) / (t_grid[-1] - t_grid[0])
    # inp = torch.stack([x_in, y_in, t_in], dim=1)
    # psi_here, p_here = model(inp)
    # ψ и p во всех нужных окрестностях
    psi_all, p_all, k_all = eval_stencil(model, i, j, k_int)
    # if first_batch:
    #     with torch.no_grad():
    #         print("psi min/max:", psi_all.min().item(), psi_all.max().item())
    #         print("p   min/max:", p_all.min().item(), p_all.max().item())
    #     first_batch = False
    # читаем по индексам (см. порядок в STENCIL)
    psi_c  = psi_all[:, 0]   # (i,j,k)
    psi_xp = psi_all[:, 1]   # (i+1,j,k)
    psi_xm = psi_all[:, 2]   # (i-1,j,k)
    psi_xpp = psi_all[:, 3]  # (i+2,j,k)
    psi_xmm = psi_all[:, 4]  # (i-2,j,k)
    psi_yp = psi_all[:, 5]
    psi_ym = psi_all[:, 6]
    psi_ypp = psi_all[:, 7]
    psi_ymm = psi_all[:, 8]
    psi_xpyp = psi_all[:, 9]
    psi_xpym = psi_all[:, 10]
    psi_xmyp = psi_all[:, 11]
    psi_xmym = psi_all[:, 12]
    psi_tp = psi_all[:, 13]
    psi_tm = psi_all[:, 14]
    psi_xp_tp = psi_all[:, 15]
    psi_xm_tp = psi_all[:, 16]
    psi_yp_tp = psi_all[:, 17]
    psi_ym_tp = psi_all[:, 18]
    psi_xp_tm = psi_all[:, 19]
    psi_xm_tm = psi_all[:, 20]
    psi_yp_tm = psi_all[:, 21]
    psi_ym_tm = psi_all[:, 22]
    p_c  = p_all[:, 0]
    p_xp = p_all[:, 1]
    p_xm = p_all[:, 2]
    p_yp = p_all[:, 5]
    p_ym = p_all[:, 6]
    # 1) скорости из ψ
    u_data = (-psi_ypp + 8*psi_yp - 8*psi_ym + psi_ymm) / (12.0 * dy)
    v_data = -(-psi_xpp + 8*psi_xp - 8*psi_xm + psi_xmm) / (12.0 * dx)
    u_pde = (psi_yp - psi_ym) / (2.0 * dy)
    v_pde = -(psi_xp - psi_xm) / (2.0 * dx)
    # соседи скоростей для НС
    u_xp = (psi_xpyp - psi_xpym) / (2.0 * dy)
    u_xm = (psi_xmyp - psi_xmym) / (2.0 * dy)
    u_yp = (psi_ypp - psi_c) / (2.0 * dy)
    u_ym = (psi_c - psi_ymm) / (2.0 * dy)
    v_xp = -(psi_xpp - psi_c) / (2.0 * dx)
    v_xm = -(psi_c - psi_xmm) / (2.0 * dx)
    v_yp = -(psi_xpyp - psi_xmyp) / (2.0 * dx)
    v_ym = -(psi_xpym - psi_xmym) / (2.0 * dx)
    # по времени
    u_tp = (psi_yp_tp - psi_ym_tp) / (2.0 * dy)
    u_tm = (psi_yp_tm - psi_ym_tm) / (2.0 * dy)
    v_tp = -(psi_xp_tp - psi_xm_tp) / (2.0 * dx)
    v_tm = -(psi_xp_tm - psi_xm_tm) / (2.0 * dx)
    du_dt = time_derivative(u_pde, u_tp, u_tm, k_int)
    dv_dt = time_derivative(v_pde, v_tp, v_tm, k_int)
    # пространственные производные
    du_dx = (u_xp - u_xm) / (2.0 * dx)
    du_dy = (u_yp - u_ym) / (2.0 * dy)
    dv_dx = (v_xp - v_xm) / (2.0 * dx)
    dv_dy = (v_yp - v_ym) / (2.0 * dy)
    lap_u = (u_xp - 2.0 * u_pde + u_xm) / (dx * dx) + (u_yp - 2.0 * u_pde + u_ym) / (dy * dy)
    lap_v = (v_xp - 2.0 * v_pde + v_xm) / (dx * dx) + (v_yp - 2.0 * v_pde + v_ym) / (dy * dy)
    dp_dx = (p_xp - p_xm) / (2.0 * dx)
    dp_dy = (p_yp - p_ym) / (2.0 * dy)
    # 2) невязки НС
    f_x = du_dt + u_pde * du_dx + v_pde * du_dy + dp_dx - (1.0 / Re) * lap_u
    f_y = dv_dt + u_pde * dv_dx + v_pde * dv_dy + dp_dy - (1.0 / Re) * lap_v
    # 3) вихрь ω = Δψ
    omega_c = -(
        (psi_xp - 2.0 * psi_c + psi_xm) / (dx * dx) 
        + (psi_yp - 2.0 * psi_c + psi_ym) / (dy * dy))
    omega_xp = -(
        (psi_xpp - 2.0 * psi_xp + psi_c) / (dx * dx) 
        + (psi_xpyp - 2.0 * psi_xp + psi_xpym) / (dy * dy))
    omega_xm = -(
        (psi_c - 2.0 * psi_xm + psi_xmm) / (dx * dx) 
        + (psi_xmyp - 2.0 * psi_xm + psi_xmym) / (dy * dy))
    omega_yp = -(
        (psi_xpyp - 2.0 * psi_yp + psi_xmyp) / (dx * dx) 
        + (psi_ypp - 2.0 * psi_yp + psi_c) / (dy * dy))
    omega_ym = -(
        (psi_xpym - 2.0 * psi_ym + psi_xmym) / (dx * dx) 
        + (psi_c - 2.0 * psi_ym + psi_ymm) / (dy * dy))
    # по времени для ω
    omega_tp = -(
        (psi_xp_tp - 2.0 * psi_tp + psi_xm_tp) / (dx * dx) 
        + (psi_yp_tp - 2.0 * psi_tp + psi_ym_tp) / (dy * dy))
    omega_tm = -(
        (psi_xp_tm - 2.0 * psi_tm + psi_xm_tm) / (dx * dx) 
        + (psi_yp_tm - 2.0 * psi_tm + psi_ym_tm) / (dy * dy))
    domega_dt = time_derivative(omega_c, omega_tp, omega_tm, k_int)
    domega_dx = (omega_xp - omega_xm) / (2.0 * dx)
    domega_dy = (omega_yp - omega_ym) / (2.0 * dy)
    lap_omega = (omega_xp - 2.0 * omega_c + omega_xm) / (dx * dx) + (omega_yp - 2.0 * omega_c + omega_ym) / (dy * dy)
    g_vort = domega_dt + u_pde * domega_dx + v_pde * domega_dy - (1.0 / Re) * lap_omega
    # сами потери
    L1 = F.mse_loss(u_data, u_true) + F.mse_loss(v_data, v_true)
    L2 = torch.mean(f_x ** 2 + f_y ** 2)
    L3 = torch.mean(g_vort ** 2)
    result = {
        "L1": L1,
        "L2": L2,
        "L3": L3,
    }
    if details:
        result.update({
            "u_data": u_data,
            "v_data": v_data,
            "u_pde": u_pde,
            "v_pde": v_pde,
            "p_pred": p_c,
            "f_x": f_x,
            "f_y": f_y,
            "g_vort": g_vort,})
    return result

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_L1 = 0.0
    running_L2 = 0.0
    running_L3 = 0.0
    n_seen = 0
    t0 = time.time()
    # прогресс по батчам
    pbar = tqdm(
        train_loader,
        total=len(train_loader),
        desc=f"epoch {epoch+1}/{num_epochs}",
        leave=False
    )
    first_batch = (epoch == 0)
    for batch in pbar:
        xb = batch["x"].to(device)
        yb = batch["y"].to(device)
        tb = batch["tau"].to(device)
        u_true = batch["u"].to(device)
        v_true = batch["v"].to(device)
        out = physics_forward(model, xb, yb, tb, u_true, v_true)
        loss = a1 * out["L1"] + a2 * out["L2"] + a3 * out["L3"]
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        bs = batch["x"].size(0)
        running_loss += loss.item() * bs
        running_L1 += out["L1"].item() * bs
        running_L2 += out["L2"].item() * bs
        running_L3 += out["L3"].item() * bs
        n_seen += bs
        pbar.set_postfix({
            "loss": f"{loss.item():.3e}",
            "L1": f"{out['L1'].item():.3e}",
            "wL2": f"{(a2*out['L2']).item():.3e}",
            "wL3": f"{(a3*out['L3']).item():.3e}",
            "L2": f"{out['L2'].item():.3e}",
            "L3": f"{out['L3'].item():.3e}",})
    # итоги по эпохе
    epoch_loss = running_loss / n_seen
    epoch_L1   = running_L1 / n_seen
    epoch_L2   = running_L2 / n_seen
    epoch_L3   = running_L3 / n_seen
    elapsed = time.time() - t0
    print(
        f"==> epoch {epoch+1:03d} done in {elapsed:.1f}s | "
        f"loss={epoch_loss:.6e} | L1={epoch_L1:.3e} | wL2={a2*epoch_L2:.3e} | wL3={a3*epoch_L3:.3e}")
    # если физика всё ещё тише, чем 1e-3, можно усилить
    if (epoch + 1 >= 5) in [10, 20, 40, 80, 120, 160]:
        if a2*epoch_L2 < 1e-3:
            a2 = min(a2 * 2.0, MAX_A2)
        # вихрь разгоняем реже и мягче
        if (epoch + 1) >= 40 and a3 * epoch_L3 < 1e-3:
            a3 = min(a3 * 2.0, MAX_A3)
    model.eval()
    val_L1 = 0.0
    n_val = 0
    with torch.no_grad():
        for batch in val_loader:
            xb = batch["x"].to(device)
            yb = batch["y"].to(device)
            tb = batch["tau"].to(device)
            u_true = batch["u"].to(device)
            v_true = batch["v"].to(device)
            out = physics_forward(model, xb, yb, tb, u_true, v_true)
            val_L1 += out["L1"].item() * xb.size(0)
            n_val += xb.size(0)
    val_L1 /= n_val
    print(f"val L1 = {val_L1:.3e}; a2={a2:.1e} | a3={a3:.1e}")


epoch 1/200:   0%|          | 0/283 [00:00<?, ?it/s]

==> epoch 001 done in 166.9s | loss=5.900918e-01 | L1=3.628e-01 | wL2=2.273e-01 | wL3=0.000e+00
val L1 = 1.475e-01; a2=1.0e-03 | a3=0.0e+00


epoch 2/200:   0%|          | 0/283 [00:00<?, ?it/s]

==> epoch 002 done in 195.4s | loss=1.194534e-01 | L1=5.971e-02 | wL2=5.974e-02 | wL3=0.000e+00
val L1 = 2.975e-02; a2=1.0e-03 | a3=0.0e+00


epoch 3/200:   0%|          | 0/283 [00:00<?, ?it/s]

...

==> epoch 198 done in 108.8s | loss=2.121510e-04 | L1=4.081e-05 | wL2=1.713e-04 | wL3=0.000e+00
val L1 = 2.649e-05; a2=1.0e-03 | a3=0.0e+00


epoch 199/200:   0%|          | 0/283 [00:00<?, ?it/s]

==> epoch 199 done in 109.1s | loss=1.609384e-03 | L1=6.405e-04 | wL2=9.689e-04 | wL3=0.000e+00
val L1 = 4.148e-03; a2=1.0e-03 | a3=0.0e+00


epoch 200/200:   0%|          | 0/283 [00:00<?, ?it/s]

==> epoch 200 done in 71.0s | loss=1.205703e-03 | L1=3.507e-04 | wL2=8.550e-04 | wL3=0.000e+00
val L1 = 3.516e-05; a2=1.0e-03 | a3=0.0e+00


In [7]:
ckpt_dir = Path("./checkpoints")
ckpt_dir.mkdir(parents=True, exist_ok=True)

def save_checkpoint(model, optimizer, scheduler, epoch, path):
    state = {
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict() if scheduler is not None else None,
    }
    torch.save(state, path)
    print(f"saved to {path}")
    
save_checkpoint(model, optimizer, scheduler, epoch+1, ckpt_dir / "tgMLP_128x128_epoch=200_a1_a2.pth",)

saved to checkpoints\tgMLP_128x128_epoch=200_a1_a2.pth


In [ ]:
def load_checkpoint(path, model, optimizer=None, scheduler=None, map_location="cpu"):
    ckpt = torch.load(path, map_location=map_location)
    model.load_state_dict(ckpt["model"])
    if optimizer is not None and "optimizer" in ckpt and ckpt["optimizer"] is not None:
        optimizer.load_state_dict(ckpt["optimizer"])
    if scheduler is not None and "scheduler" in ckpt and ckpt["scheduler"] is not None:
        scheduler.load_state_dict(ckpt["scheduler"])
    print(f"loaded from {path}, epoch={ckpt.get('epoch', '?')}")
    return ckpt.get("epoch", None)

state_dict = torch.load("tgMLP_64x64_epoch=200.pth", map_location="cpu")
model.load_state_dict(state_dict)
model.to(device)
model.eval()

In [8]:
model.eval()

sum_u_mse = 0.0
sum_v_mse = 0.0
sum_p_mse_raw = 0.0
sum_u_true_sq = 0.0
sum_v_true_sq = 0.0

sum_dp = 0.0 # Σ (p_pred - p_true)
sum_dp2 = 0.0 # Σ (p_pred - p_true)^2

sum_fx_mse = 0.0
sum_fy_mse = 0.0
sum_g_mse  = 0.0

max_fx = 0.0
max_fy = 0.0
max_g  = 0.0

n_points = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc="eval on test", leave=False):
        xb = batch["x"].to(device)
        yb = batch["y"].to(device)
        tb = batch["tau"].to(device)
        u_true = batch["u"].to(device)
        v_true = batch["v"].to(device)
        p_true = batch["p"].to(device)
        B = xb.size(0)
        out = physics_forward(
            model,
            xb, yb, tb,
            u_true=u_true,
            v_true=v_true,
            details=True)
        # данные
        u_pred = out["u_data"] # 4-й порядок, как в train
        v_pred = out["v_data"]
        p_pred = out["p_pred"]
        # физика
        f_x = out["f_x"]
        f_y = out["f_y"]
        g_vort = out["g_vort"]
        # метрики по данным
        mse_u = torch.mean((u_pred - u_true) ** 2)
        mse_v = torch.mean((v_pred - v_true) ** 2)
        mse_p_raw = torch.mean((p_pred - p_true) ** 2)
        sum_u_mse += mse_u.item() * B
        sum_v_mse += mse_v.item() * B
        sum_p_mse_raw += mse_p_raw.item() * B
        # для относительных норм
        sum_u_true_sq += torch.mean(u_true ** 2).item() * B
        sum_v_true_sq += torch.mean(v_true ** 2).item() * B
        # shift-invariant давление
        # d = p_pred - p_true
        dp = (p_pred - p_true)
        sum_dp  += dp.sum().item()
        sum_dp2 += (dp * dp).sum().item()
        # метрики по физике
        mse_fx = torch.mean(f_x ** 2)
        mse_fy = torch.mean(f_y ** 2)
        mse_g  = torch.mean(g_vort ** 2)
        sum_fx_mse += mse_fx.item() * B
        sum_fy_mse += mse_fy.item() * B
        sum_g_mse  += mse_g.item()  * B
        max_fx = max(max_fx, torch.max(torch.abs(f_x)).item())
        max_fy = max(max_fy, torch.max(torch.abs(f_y)).item())
        max_g  = max(max_g,  torch.max(torch.abs(g_vort)).item())
        n_points += B

# усреднение
mean_mse_u = sum_u_mse / n_points
mean_mse_v = sum_v_mse / n_points
mean_mse_p_raw = sum_p_mse_raw / n_points

rel_l2_u = math.sqrt(mean_mse_u) / math.sqrt(sum_u_true_sq / n_points + 1e-12)
rel_l2_v = math.sqrt(mean_mse_v) / math.sqrt(sum_v_true_sq / n_points + 1e-12)

mean_mse_fx = sum_fx_mse / n_points
mean_mse_fy = sum_fy_mse / n_points
mean_mse_g  = sum_g_mse  / n_points

# shift-invariant давление
# d̄ = mean(p_pred - p_true)
mean_dp  = sum_dp / n_points
mean_dp2 = sum_dp2 / n_points
# MSE после вычитания оптимальной константы
mse_p_shift = mean_dp2 - mean_dp ** 2
mse_p_shift = max(mse_p_shift, 0.0) # на всякий случай от численных артефактов


print("=== Test metrics ===")
print(f"MSE(u): {mean_mse_u:.6e}")
print(f"MSE(v): {mean_mse_v:.6e}")
print(f"MSE(p̂ vs p) RAW: {mean_mse_p_raw:.6e}")
print(f"MSE(p̂ vs p) shift-invariant: {mse_p_shift:.6e}")
print(f"rel L2(u): {rel_l2_u:.6e}")
print(f"rel L2(v): {rel_l2_v:.6e}")
print()
print("Physics residuals (RMS):")
print(f"NS fx RMS: {math.sqrt(mean_mse_fx):.6e}")
print(f"NS fy RMS: {math.sqrt(mean_mse_fy):.6e}")
print(f"Vorticity RMS: {math.sqrt(mean_mse_g):.6e}")
print()
print("Physics residuals (max over test):")
print(f"max |fx|: {max_fx:.6e}")
print(f"max |fy|: {max_fy:.6e}")
print(f"max |g_vort|: {max_g:.6e}")
print(f"optimal pressure shift c* = {-mean_dp:.6e}")

eval on test:   0%|          | 0/61 [00:00<?, ?it/s]

=== Test metrics ===
MSE(u): 1.538861e-05
MSE(v): 1.998927e-05
MSE(p̂ vs p) RAW: 2.018511e-01
MSE(p̂ vs p) shift-invariant: 2.237757e-04
rel L2(u): 9.187068e-03
rel L2(v): 1.045586e-02

Physics residuals (RMS):
NS fx RMS: 9.679624e-02
NS fy RMS: 1.771966e-01
Vorticity RMS: 1.720022e+01

Physics residuals (max over test):
max |fx|: 1.344740e+00
max |fy|: 2.575957e+00
max |g_vort|: 2.565495e+02
optimal pressure shift c* = -4.490294e-01
